In [1]:
import tkinter as tk
from tkinter import ttk, filedialog
from subprocess import call
from PIL import Image, ImageTk
# from PIL import Image
from tkinter import messagebox as ms
# import imageio
import cv2
import numpy as np
from scipy import ndimage  # To call your Jupyter Notebook
import pandas as pd
import numpy as np
import cv2
from matplotlib import pyplot as plt
import math
# import imutils
from matplotlib.offsetbox import AnchoredText
from sklearn import preprocessing
import seaborn as sns
from sklearn import metrics
from sklearn import datasets
from six import StringIO  
# from IPython.display import Image  
from sklearn.tree import export_graphviz
# import pydotplus
from scipy.spatial import distance
import nbimporter
from detect import detectFeatures


In [2]:
filename = ""
fileName = 'path_to_your_file'
selected_model = None

root = tk.Tk()
root.configure(background="seashell2")
w, h = root.winfo_screenwidth(), root.winfo_screenheight()
root.geometry("%dx%d+0+0" % (w, h))
root.title("Melanoma Cancer Detection using Deep Learning")

lbl = tk.Label(root, text="Melanoma Cancer Detection using Deep Learning", font=('times', 35,' bold '), height=1, width=65,bg="violet Red",fg="Black",anchor='n',justify='center')
lbl.place(x=0, y=0)

In [3]:
image_labels = []

In [4]:
import os,glob
threshold = 149.0
def openimage():
   
    global fn
    global image_labels
    for label in image_labels:
        label.destroy()
    files = glob.glob('processed_img' + '/*')
    for f in files:
        os.remove(f)
    image_labels = []
    f_types = [('Image Files', '*.jpg'), ('PNG Files', '*.png'), ('JPEG Files', '*.jpeg')]
    fileName = filedialog.askopenfilename(filetypes=f_types)
    IMAGE_SIZE=200
    imgpath = fileName
    fn = fileName


#        img = Image.open(imgpath).convert("L")
    img = Image.open(imgpath)
    
    img = img.resize((IMAGE_SIZE,200))
    img = np.array(img)
#        img = img / 255.0
#        img = img.reshape(1,IMAGE_SIZE,IMAGE_SIZE,3)


    x1 = int(img.shape[0])
    y1 = int(img.shape[1])


#
#        gs = cv2.cvtColor(cv2.imread(imgpath, 1), cv2.COLOR_RGB2GRAY)
#
#        gs = cv2.resize(gs, (x1, y1))
#
#        retval, threshold = cv2.threshold(gs, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    im = Image.fromarray(img)
    imgtk = ImageTk.PhotoImage(im)
    img = tk.Label(root, image=imgtk, height=200, width=200)
    
    #result_label1 = tk.Label(root, image=imgtk, width=250,height=250)
    #result_label1.place(x=300, y=100)
    header = tk.Label(root, text="Selected Image", font=('times', 15, 'bold'), height=1, width=16)
    header.place(x=303, y=170)  # Adjust the y-coordinate as needed
    image_labels.append(header)
    img.image = imgtk
    img.place(x=300, y=220)
    image_labels.append(img)
    
def remove_hairs_im(image):
    grayScale = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (17, 17))
    blackhat = cv2.morphologyEx(grayScale, cv2.MORPH_BLACKHAT, kernel)
    _, thresh2 = cv2.threshold(blackhat, 10, 255, cv2.THRESH_BINARY)
    dst = cv2.inpaint(image, thresh2, 1, cv2.INPAINT_TELEA)
    return dst

def remove_hairs_img():
    global fn
    if fn:
        img = cv2.imread(fn)
        processed_img = remove_hairs_im(img)
        cv2.imwrite('processed_img/remove_hairs.jpg',processed_img)
        processed_img = cv2.cvtColor(processed_img, cv2.COLOR_BGR2RGB)  # Convert image to RGB for PIL
        # cv2.imwrite(r'C:\Users\AJAY\Desktop\melanoma-detection-main\processed_img\remove_hairs.jpg',processed_img)
        processed_img = Image.fromarray(processed_img)
        processed_img = ImageTk.PhotoImage(processed_img)
        header = tk.Label(root, text="Hair Removed Image", font=('times', 15, 'bold'), height=1, width=16)
        header.place(x=553, y=170)  # Adjust the y-coordinate as needed
        image_labels.append(header)
        img3 = tk.Label(root, image=processed_img, height=200, width=200)
        img3.image = processed_img
        img3.place(x=550, y=220)
        image_labels.append(img3)
        
    else:
        print("Please select an image first.")        

def convertToGreyScale_im(image):
    image = remove_hairs_im(image)
    def getWeightedAvg(pixel):
        return 0.299 * pixel[0] + 0.587 * pixel[1] + 0.114 * pixel[2]        
    grey = np.zeros(image.shape[0:-1])
    for rownum in range(len(image)):
        for colnum in range(len(image[rownum])):
            grey[rownum][colnum] = getWeightedAvg(image[rownum][colnum])        
    return grey

def convertToGreyScale_img():
    global fn
    if fn:
        img = cv2.imread(fn)
        processed_img = convertToGreyScale_im(img)
        cv2.imwrite('processed_img/grey_scale.jpg',processed_img)
        # processed_img = cv2.cvtColor(processed_img, cv2.COLOR_BGR2RGB)  # Convert image to RGB for PIL
        processed_img = Image.fromarray(processed_img)
        processed_img = ImageTk.PhotoImage(processed_img)
        header = tk.Label(root, text="Grey Scaled Image", font=('times', 15, 'bold'), height=1, width=16)
        header.place(x=803, y=170)  # Adjust the y-coordinate as needed
        image_labels.append(header)
        img3 = tk.Label(root, image=processed_img, height=200, width=200)
        img3.image = processed_img
        img3.place(x=800, y=220)
        image_labels.append(img3)
        
    else:
        print("Please select an image first.")

def threshold_im(image, threshold):
    assert len(image.shape) == 2, "Must be grayscale image"
    thresh = np.zeros(image.shape)
    for rownum in range(len(image)):
        for colnum in range(len(image[rownum])):    
            if image[rownum][colnum] > threshold:
                thresh[rownum][colnum] = 0 
            else:
                thresh[rownum][colnum] = 255
    return thresh

def threshold_img():
    global fn
    if fn:
        img = cv2.imread(fn)
        processed_img = threshold_im(convertToGreyScale_im(img),threshold)
        # processed_img = cv2.cvtColor(processed_img, cv2.COLOR_BGR2RGB)  # Convert image to RGB for PIL
        
        processed_img = Image.fromarray(processed_img)
        processed_img = ImageTk.PhotoImage(processed_img)

        img4 = tk.Label(root, image=processed_img, height=200, width=200)
        img4.image = processed_img
        img4.place(x=1200, y=200)
        image_labels.append(img4)
        
    else:
        print("Please select an image first.") 

def getHistGray_im(image):
    assert len(image.shape) == 2, "Must be grayscale image"
    hist = np.zeros(255)
    for row in image:
        for col in row:
            hist[int(col)] += 1
    return hist

def getHistGray_img():
    global fn
    if fn:
        img = cv2.imread(fn)
        processed_img = getHistGray_im(convertToGreyScale_im(img))
        # processed_img = cv2.cvtColor(processed_img, cv2.COLOR_BGR2RGB)  # Convert image to RGB for PIL
        processed_img = Image.fromarray(processed_img)
        processed_img = ImageTk.PhotoImage(processed_img)

        img4 = tk.Label(root, image=processed_img, height=200, width=200)
        img4.image = processed_img
        img4.place(x=600, y=400)
        image_labels.append(img4)
        
    else:
        print("Please select an image first.") 

def otsu_im(image):
    assert len(image.shape) == 2, "Must be grayscale image"
    th = _getOtsuThreshold_im(image)
    return threshold_im(image, th)

def otsu_img():
    global fn
    if fn:
        img = cv2.imread(fn)
        processed_img = otsu_im(convertToGreyScale_im(img))
        cv2.imwrite('processed_img/threshold.jpg',processed_img)
        # processed_img = cv2.cvtColor(processed_img, cv2.COLOR_BGR2RGB)  # Convert image to RGB for PIL
        processed_img = Image.fromarray(processed_img)
        processed_img = ImageTk.PhotoImage(processed_img)
        header = tk.Label(root, text="Thresholded Image", font=('times', 15, 'bold'), height=1, width=16)
        header.place(x=1053, y=170)  # Adjust the y-coordinate as needed
        image_labels.append(header)
        img4 = tk.Label(root, image=processed_img, height=200, width=200)
        img4.image = processed_img
        img4.place(x=1050, y=220)
        image_labels.append(img4)
        
    else:
        print("Please select an image first.")     
    
def _getOtsuThreshold_im(image):
    s = 0
    histogram = getHistGray_im(image)
    for i in range(len(histogram)):
        s += i * histogram[i]
    sumB = 0
    wB = 0
    wF = 0
    mB = None
    mF = None
    m = 0.0
    between = 0.0
    threshold1 = 0.0
    threshold2 = 0.0
    total = len(image.ravel())
    for i in range(len(histogram)):
        wB += histogram[i]
        if wB == 0:
            continue
        wF = total - wB
        if wF == 0:
            break
        sumB += i * histogram[i]
        mB = sumB / wB
        mF = (s - sumB) / wF
        between = wB * wF * ((mB - mF) ** 2)
        if between >= m:
            threshold1 = i
            if between > m:
                threshold2 = i
            m = between
    return (threshold1 + threshold2) / 2.0

def masking_im(image, mask):
    new_img = np.array(image)
    new_img = remove_hairs_im(new_img)
    for row in range(len(image)):
        for col in range(len(image[row])):
            if mask[row,col]==0:
                new_img[row,col] = 0
            if mask[row,col]==1:
                new_img[row,col] = image[row,col]
    return new_img

def masking_img():
    global fn
    if fn:
        img = cv2.imread(fn)
        processed_img = remove_hairs_im(masking_im(img,opened_im(img)))
        cv2.imwrite('processed_img/final_img.jpg',processed_img)
        processed_img = cv2.cvtColor(processed_img, cv2.COLOR_BGR2RGB)  # Convert image to RGB for PIL
        processed_img = Image.fromarray(processed_img)
        processed_img = ImageTk.PhotoImage(processed_img)
        header = tk.Label(root, text="Final Image", font=('times', 15, 'bold'), height=1, width=16)
        header.place(x=1303, y=170)  # Adjust the y-coordinate as needed
        image_labels.append(header)
        img4 = tk.Label(root, image=processed_img, height=200, width=200)
        img4.image = processed_img
        img4.place(x=1300, y=220)
        image_labels.append(img4)
        
    else:
        print("Please select an image first.") 

def opened_im(image):
    grey = convertToGreyScale_im(image)
    thresh = otsu_im(grey)
    dilimg = ndimage.binary_dilation(thresh)
    for x in range(25):
        dilimg = ndimage.binary_dilation(dilimg)
    erimg = ndimage.binary_erosion(dilimg)
    for x in range(25):
        erimg = ndimage.binary_erosion(erimg)
    return erimg

def opened_img():
    global fn
    if fn:
        img = cv2.imread(fn)
        processed_img = opened_im(img)
        # processed_img = cv2.cvtColor(processed_img, cv2.COLOR_BGR2RGB)  # Convert image to RGB for PIL
        processed_img = Image.fromarray(processed_img)
        processed_img = ImageTk.PhotoImage(processed_img)

        img4 = tk.Label(root, image=processed_img, height=200, width=200)
        img4.image = processed_img
        img4.place(x=600, y=400)
        image_labels.append(img4)
        
    else:
        print("Please select an image first.")        

def detectFeature():
    try:
        # fn = r'C:/Users\AJAY\Desktop\melanoma-detection-main\processed_img\final_img.jpg'
        # img = cv2.imread(J)
        global fn
        img = fn
        status, level = detectFeatures(img)
        if(status == 'Negative'):
            level = 100 - level
        level = round(level,2)
        my_font1 = ('times', 15, 'bold')
        l2 = tk.Label(root,text='Melanoma Diagnosis : '+status,width=80,font=my_font1)  
        l2.place(x=300,y=500)
        image_labels.append(l2)
        l3 = tk.Label(root,text='Confidence Value : '+str(level)+'%',width=80,font=my_font1)  
        l3.place(x=300,y=550)
        image_labels.append(l3)
    except Exception as e:
        print(e)
        ms.showerror('Error', 'Error in detecting features. Please try again.')


In [5]:
frame_alpr = tk.LabelFrame(root, text=" --Process-- ", width=220, height=300, bd=5, font=('times', 14, ' bold '),bg="lawn Green")
frame_alpr.grid(row=0, column=0, sticky='nw')
frame_alpr.place(x=40, y=150)

button1 = tk.Button(frame_alpr, text=" Select_Image ", command=openimage,width=15, height=1, font=('times', 15, ' bold '),bg="white",fg="black")
button1.place(x=10, y=20)

remove_hairs_button = tk.Button(frame_alpr, text="Remove Hairs", command=remove_hairs_img,width=15, height=1, font=('times', 15, ' bold '),bg="white",fg="black")
remove_hairs_button.place(x=10,y=70)

greyscale_button = tk.Button(frame_alpr, text="Grey Scale", command=convertToGreyScale_img,width=15, height=1, font=('times', 15, ' bold '),bg="white",fg="black")
greyscale_button.place(x=10,y=120)

otsu_img_button = tk.Button(frame_alpr, text="Threshold", command=otsu_img,width=15, height=1, font=('times', 15, ' bold '),bg="white",fg="black")
otsu_img_button.place(x=10,y=170)

final_img_button = tk.Button(frame_alpr, text="Final Image", command=masking_img,width=15, height=1, font=('times', 15, ' bold '),bg="white",fg="black")
final_img_button.place(x=10,y=220)

frame_alpr_model = tk.LabelFrame(root, text=" --Analysis-- ", width=220, height=120, bd=5, font=('times', 14, ' bold '),bg="lawn Green")
frame_alpr_model.grid(row=0, column=0, sticky='nw')
frame_alpr_model.place(x=40, y=500)

detect_button = tk.Button(frame_alpr_model, text="Detect Features", command=detectFeature,width=15, height=1, font=('times', 15, ' bold '),bg="white",fg="black")
detect_button.place(x=10,y=20)
# Run the main loop
root.mainloop()

c:\Users\AJAY\Desktop\melanoma-diagnosis-analysis-main\.conda\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.23.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
c:\Users\AJAY\Desktop\melanoma-diagnosis-analysis-main\.conda\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator SVC from version 0.23.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
c:\Users\AJAY\Desktop\melanoma-diagnosis-analysis-main\.conda\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator KNeighborsClassifier from version 0.23.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
c:\Users\AJAY\Desktop\melanoma-diagnosis-analysis-main\.conda\lib\site-packages\sklearn\base.py:310: UserWarni

In [ ]:
# import tkinter as tk
# from tkinter import filedialog
# from tkinter.filedialog import askopenfile
# from PIL import Image, ImageTk
# my_w = tk.Tk()
# my_w.geometry("750x600") 
# my_w.title('Melanoma Detection')
# my_font1=('times', 18, 'bold')
# l1 = tk.Label(my_w,text='Upload Image of the Skin Lesion',width=80,font=my_font1)  
# l1.grid(row=1,column=1)
# b1 = tk.Button(my_w, text='Upload File', width=20,command = lambda:upload_file())
# b1.grid(row=2,column=1) 

# def upload_file():
#     global img
#     f_types = [('Image Files', '*.jpg'), ('PNG Files', '*.png'), ('JPEG Files', '*.jpeg')]
#     filename = filedialog.askopenfilename(filetypes=f_types)
#     print(filename)
#     print("Image Uploaded")
#     status, level = detectFeatures(filename)
#     print(status, level)
#     if(status == 'Negative'):
#         level = 100 - level
#     level = round(level,2)
#     img = ImageTk.PhotoImage(file=filename)
#     b2 =tk.Button(my_w,image=img) 
#     b2.grid(row=3,column=1)
#     l2 = tk.Label(my_w,text='Melanoma Diagnosis : '+status,width=80,font=my_font1)  
#     l2.grid(row=4,column=1)
#     l3 = tk.Label(my_w,text='Confidence Value : '+str(level)+'%',width=80,font=my_font1)  
#     l3.grid(row=5,column=1)
#     # except:
#     print("No Image")

# my_w.mainloop()